In [22]:
import pandas as pd
import numpy as np
import re
import unicodedata

In [4]:
csv_path = "../data/raw/immobilier.csv"
df = pd.read_csv(csv_path)
df

C:\Users\MSI\AppData\Local\Temp\ipykernel_25428\2048431866.py:2: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)


,id,title,slug,createdAt,price,priceUnit,area,city,store
0,51228043,Vente Appartement F3 F4 Alger Kouba,appartement-vente-f3-f4-alger-kouba-algerie,2025-12-10T17:24:43.000Z,33700000.0,MILLION,['114 m²'],Kouba,Groupe Ahmed Soltan
1,40312882,Vente Appartement F3 Alger Bordj el kiffan,appartement-vente-f3-alger-bordj-el-kiffan-alg...,2025-12-10T17:24:42.000Z,12200000.0,MILLION,['76 m²'],Bordj el kiffan,Groupe Ahmed Soltan
2,35052897,Vente Appartement F3 Alger Bordj el kiffan,appartement-vente-f3-alger-bordj-el-kiffan-alg...,2025-12-10T17:24:36.000Z,NaN,UNIT,['81 m²'],Bordj el kiffan,Dream home
3,51245359,Vente Appartement F5 Alger Cheraga,appartement-vente-f5-alger-cheraga-algerie,2025-12-10T17:24:28.000Z,NaN,BILLION,['177 m²'],Cheraga,Aghiles Promotion Ikram
4,50817718,Vente Appartement F2 F3 F4 Oran Bir el djir,appartement-vente-f2-f3-f4-oran-bir-el-djir-al...,2025-12-10T17:24:26.000Z,180000.0,MILLION,NaN,Bir el djir,Valor Promotion immobilière
...,...,...,...,...,...,...,...,...,...
184529,24652026,Vente Terrain Msila Bou saada,msila-bou-saada-algerie-terrain-vente,2020-11-20T08:59:24.000Z,3000000.0,MILLION,NaN,Bou saada,NaN
184530,10845427,Vente Terrain Batna Batna,batna-algerie-terrain-vente,2020-11-20T01:08:41.000Z,0.0,MILLION,NaN,Batna,NaN
184531,10845447,Vente Terrain Batna Batna,batna-algerie-terrain-vente,2020-11-20T01:08:37.000Z,0.0,MILLION,NaN,Batna,NaN
184532,24729275,Vente Appartement F4 Sidi bel abbes Sidi bel a...,sidi-bel-abbes-algerie-appartement-vente-f4,2020-11-20T00:58:59.000Z,9500000.0,MILLION,NaN,Sidi bel abbes,NaN


### Drop unnecessary columns :
##### id, title ( holds same value as slug ), 

In [5]:
df.drop(columns=["title", "id"], inplace=True)

## **Handle Deal type (Sale vs Rent) :** 
##### After inspecting data in data wrangler we notice that the column "title" has a distingished deal type , "vente" representing a sale listing , and "location" representing a rent listing , code below will create a new column "deal type" that will handle this

In [9]:
# 1. adding a deal type column

def extract_deal_type(slug):
    if not isinstance(slug, str):
        return "unknown"
    
    t = slug.lower()

    if "vente" in t:
        return "vente"
    if "location" in t:
        return "location"
    
    return "unknown"

df["dealType"] = df["slug"].apply(extract_deal_type)


In [11]:
df[(["slug", "dealType"])]

,slug,dealType
0,appartement-vente-f3-f4-alger-kouba-algerie,vente
1,appartement-vente-f3-alger-bordj-el-kiffan-alg...,vente
2,appartement-vente-f3-alger-bordj-el-kiffan-alg...,vente
3,appartement-vente-f5-alger-cheraga-algerie,vente
4,appartement-vente-f2-f3-f4-oran-bir-el-djir-al...,vente
...,...,...
184529,msila-bou-saada-algerie-terrain-vente,vente
184530,batna-algerie-terrain-vente,vente
184531,batna-algerie-terrain-vente,vente
184532,sidi-bel-abbes-algerie-appartement-vente-f4,vente


### **Extract numeric area ( "120 m²" → 120 )**

In [ ]:

def clean_area(x):
    if pd.isna(x):
        return np.nan
    m = re.search(r"(\d+)", str(x))
    return float(m.group(1)) if m else np.nan

df["area_m2"] = df["area"].apply(clean_area)
df = df.drop(columns=["area"])

### **Extract useful time-based features for analysis:**

In [ ]:
df["createdAt"] = pd.to_datetime(df["createdAt"], errors="coerce")
df['created_at_dt'] = pd.to_datetime(df['createdAt'])
df['listing_year'] = df['created_at_dt'].dt.year
df['listing_month'] = df['created_at_dt'].dt.month
df['listing_dayofweek'] = df['created_at_dt'].dt.dayofweek
df['listing_hour'] = df['created_at_dt'].dt.hour
df = df.drop(columns=['createdAt'])

In [15]:
df

,slug,price,priceUnit,city,store,dealType,area_m2,created_at_dt,listing_year,listing_month,listing_dayofweek,listing_hour
0,appartement-vente-f3-f4-alger-kouba-algerie,33700000.0,MILLION,Kouba,Groupe Ahmed Soltan,vente,114.0,2025-12-10 17:24:43+00:00,2025,12,2,17
1,appartement-vente-f3-alger-bordj-el-kiffan-alg...,12200000.0,MILLION,Bordj el kiffan,Groupe Ahmed Soltan,vente,76.0,2025-12-10 17:24:42+00:00,2025,12,2,17
2,appartement-vente-f3-alger-bordj-el-kiffan-alg...,NaN,UNIT,Bordj el kiffan,Dream home,vente,81.0,2025-12-10 17:24:36+00:00,2025,12,2,17
3,appartement-vente-f5-alger-cheraga-algerie,NaN,BILLION,Cheraga,Aghiles Promotion Ikram,vente,177.0,2025-12-10 17:24:28+00:00,2025,12,2,17
4,appartement-vente-f2-f3-f4-oran-bir-el-djir-al...,180000.0,MILLION,Bir el djir,Valor Promotion immobilière,vente,NaN,2025-12-10 17:24:26+00:00,2025,12,2,17
...,...,...,...,...,...,...,...,...,...,...,...,...
184529,msila-bou-saada-algerie-terrain-vente,3000000.0,MILLION,Bou saada,NaN,vente,NaN,2020-11-20 08:59:24+00:00,2020,11,4,8
184530,batna-algerie-terrain-vente,0.0,MILLION,Batna,NaN,vente,NaN,2020-11-20 01:08:41+00:00,2020,11,4,1
184531,batna-algerie-terrain-vente,0.0,MILLION,Batna,NaN,vente,NaN,2020-11-20 01:08:37+00:00,2020,11,4,1
184532,sidi-bel-abbes-algerie-appartement-vente-f4,9500000.0,MILLION,Sidi bel abbes,NaN,vente,NaN,2020-11-20 00:58:59+00:00,2020,11,4,0


### **Extract property type from slug:
##### Since we don't know how many types and their category exist in the dataset , we will extract them all, save them in a df and analyse the most occured ones to build a helper function to annotate the original df correctly , since "slug" structure is not consistent ( property type is not always the first word so better to look it up after each hiphen (-))

In [ ]:
# We assume Property type = first word in slug
df['property_type'] = df['slug'].str.split('-').str[0].str.lower()
# 3. Count the occurrences and sort them in descending order
property_type_counts = df['property_type'].value_counts()
property_type_df = property_type_counts.reset_index(name='occurance').rename(columns={'index':'type'})
# 4. Display the results
print("--- Top 20 Most Frequent property types in the 'slug' Column ---")
print(property_type_df.head(20))

--- Top 20 Most Frequent property types in the 'slug' Column ---
   property_type  occurance
0    appartement      75613
1          villa      28122
2        terrain      23738
3          local      20211
4          alger       4456
5         hangar       4040
6          autre       2627
7       immeuble       2578
8         niveau       1908
9           oran       1791
10         blida       1177
11       tlemcen       1094
12     boumerdes        952
13      location        927
14      carcasse        904
15        tipaza        891
16         vente        732
17   constantine        730
18           ain        714
19        bejaia        698


In [17]:
property_type_df.to_csv('../data/processed/property_type_counts.csv', index=False)
print("✓ Saved to ../data/processed/property_type_counts.csv")

✓ Saved to ../data/processed/property_type_counts.csv


In [23]:
# Property type mapping for direct hits and keyword search
DIRECT_PROPERTY_MAP = {
    'appartement': 'Apartment', 'studio': 'Apartment', 'villa': 'Villa', 
    'terrain': 'Land', 'local': 'Commercial', 'hangar': 'Industrial/Hangar', 
    'immeuble': 'Building', 'niveau': 'House/Duplex', 'duplex': 'House/Duplex', 
    'carcasse': 'Carcasse', 'bungalow': 'Bungalow', 'usine': 'Industrial/Hangar'
}

# Ambiguous words that trigger a full slug search (mostly cities and transactions)
AMBIGUOUS_FIRST_WORDS = [
    'alger', 'oran', 'blida', 'tlemcen', 'boumerdes', 'location', 'tipaza', 
    'vente', 'constantine', 'ain', 'bejaia', 'mostaganem', 'batna', 'setif', 
    'tizi', 'jijel', 'sidi', 'chlef', 'bordj', 'annaba', 'biskra', 'mascara', 
    'bouira', 'el', 'medea', 'tiaret', 'skikda', 'djelfa', 'oum', 'relizane', 
    'tebessa', 'mila', 'khenchela', 'souk', 'msila', 'bechar', 'guelma', 
    'cherche', 'saida', 'laghouat', 'ouargla', 'ghardaia', 'tissemsilt', 
    'naama', 'etranger', 'adrar', 'echange', 'tamanrasset', 'illizi', 
    'immobilier', 'tindouf', 'autre'
]

SEARCH_KEYWORDS = list(DIRECT_PROPERTY_MAP.keys())
# Regex pattern to find any property keyword surrounded by word boundaries
TYPE_PATTERN = r'\b(' + '|'.join(re.escape(k) for k in SEARCH_KEYWORDS) + r')\b'

def extract_property_type_robust(slug):
    """Extract property type using two-stage (first word then full slug search) logic."""
    if pd.isna(slug):
        return 'Unknown'
    
    slug_lower = str(slug).lower()
    first_word = slug_lower.split('-')[0]
    
    # 1. Direct Hit Check
    if first_word in DIRECT_PROPERTY_MAP:
        return DIRECT_PROPERTY_MAP[first_word]
    
    # 2. Ambiguous/Location First Word Check (Full Slug Search)
    if first_word in AMBIGUOUS_FIRST_WORDS:
        slug_space = slug_lower.replace('-', ' ')

        # Search for any property keyword
        type_match = re.search(TYPE_PATTERN, slug_space)
        if type_match:
            matched_keyword = type_match.group(1)
            return DIRECT_PROPERTY_MAP.get(matched_keyword, 'Uncategorized')
        
    return 'Uncategorized'
df['property_type'] = df['slug'].apply(extract_property_type_robust)


### **Annotate Province name and code number from slug column** 

In [24]:

# 1. Normalize text (remove accents, lowercase, hyphens)
def normalize(text):
    if pd.isna(text):
        return ""
    # remove accents
    text = ''.join(
        c for c in unicodedata.normalize('NFD', text)
        if unicodedata.category(c) != 'Mn'
    )
    text = text.lower()
    # remove apostrophes, replace spaces with hyphens for robust matching
    text = re.sub(r"[’']", "", text)
    text = text.replace(" ", "-")
    return text



# 2. Build wilaya lookup table (code + name + normalized)
wilayas = [
    (1, "Adrar"), (2, "Chlef"), (3, "Laghouat"), (4, "Oum El Bouaghi"), (5, "Batna"),
    (6, "Bejaia"), (7, "Biskra"), (8, "Bechar"), (9, "Blida"), (10, "Bouira"),
    (11, "Tamanrasset"), (12, "Tebessa"), (13, "Tlemcen"), (14, "Tiaret"),
    (15, "Tizi Ouzou"), (16, "Alger"), (17, "Djelfa"), (18, "Jijel"), (19, "Setif"),
    (20, "Saida"), (21, "Skikda"), (22, "Sidi Bel Abbes"), (23, "Annaba"),
    (24, "Guelma"), (25, "Constantine"), (26, "Medea"), (27, "Mostaganem"),
    (28, "MSila"), (29, "Mascara"), (30, "Ouargla"), (31, "Oran"),
    (32, "El Bayadh"), (33, "Illizi"), (34, "Bordj Bou Arreridj"),
    (35, "Boumerdes"), (36, "El Tarf"), (37, "Tindouf"), (38, "Tissemsilt"),
    (39, "El Oued"), (40, "Khenchela"), (41, "Souk Ahras"), (42, "Tipaza"),
    (43, "Mila"), (44, "Ain Defla"), (45, "Naama"), (46, "Ain Temouchent"),
    (47, "Ghardaia"), (48, "Relizane"), (49, "Timimoun"),
    (50, "Bordj Badji Mokhtar"), (51, "Ouled Djellal"), (52, "Beni Abbes"),
    (53, "In Salah"), (54, "In Guezzam"), (55, "Touggourt"), (56, "Djanet"),
    (57, "El MGhair"), (58, "El Meniaa"), (59, "Aflou"), (60, "Barika"),
    (61, "Ksar Chellala"), (62, "Messaad"), (63, "Ain Oussera"),
    (64, "Boussaada"), (65, "El Abiodh Sidi Cheikh"), (66, "El Kantara"),
    (67, "Bir El Ater"), (68, "Ksar El Boukhari"), (69, "El Aricha"),
]

wilaya_df = pd.DataFrame(wilayas, columns=["code", "name"])
wilaya_df["name_norm"] = wilaya_df["name"].apply(normalize)


# 3. Detect wilaya in slug (longest-name wins)

def extract_wilaya(slug):
    slug_norm = normalize(slug)

    matches = []
    for _, row in wilaya_df.iterrows():
        name_norm = row["name_norm"]

        # strict boundary matching:
        # wilaya words must appear as hyphen-separated tokens in the slug
        pattern = fr"(^|-)({name_norm})(-|$)"
        if re.search(pattern, slug_norm):
            matches.append((len(name_norm), row["name"], row["code"]))

    if not matches:
        return pd.Series([None, None])

    # pick the wilaya with the longest name (avoid partial matches)
    best = max(matches, key=lambda x: x[0])
    return pd.Series([best[1], best[2]])


# 4. Apply to df

df[["wilaya", "wilaya_code"]] = df["slug"].apply(extract_wilaya)


In [26]:
df["wilaya_code"] = df["wilaya_code"].astype("Int64")
df

,slug,price,priceUnit,city,store,dealType,area_m2,created_at_dt,listing_year,listing_month,listing_dayofweek,listing_hour,property_type_robust,property_type,wilaya,wilaya_code
0,appartement-vente-f3-f4-alger-kouba-algerie,33700000.0,MILLION,Kouba,Groupe Ahmed Soltan,vente,114.0,2025-12-10 17:24:43+00:00,2025,12,2,17,Apartment,Apartment,Alger,16
1,appartement-vente-f3-alger-bordj-el-kiffan-alg...,12200000.0,MILLION,Bordj el kiffan,Groupe Ahmed Soltan,vente,76.0,2025-12-10 17:24:42+00:00,2025,12,2,17,Apartment,Apartment,Alger,16
2,appartement-vente-f3-alger-bordj-el-kiffan-alg...,NaN,UNIT,Bordj el kiffan,Dream home,vente,81.0,2025-12-10 17:24:36+00:00,2025,12,2,17,Apartment,Apartment,Alger,16
3,appartement-vente-f5-alger-cheraga-algerie,NaN,BILLION,Cheraga,Aghiles Promotion Ikram,vente,177.0,2025-12-10 17:24:28+00:00,2025,12,2,17,Apartment,Apartment,Alger,16
4,appartement-vente-f2-f3-f4-oran-bir-el-djir-al...,180000.0,MILLION,Bir el djir,Valor Promotion immobilière,vente,NaN,2025-12-10 17:24:26+00:00,2025,12,2,17,Apartment,Apartment,Oran,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184529,msila-bou-saada-algerie-terrain-vente,3000000.0,MILLION,Bou saada,NaN,vente,NaN,2020-11-20 08:59:24+00:00,2020,11,4,8,Land,Land,MSila,28
184530,batna-algerie-terrain-vente,0.0,MILLION,Batna,NaN,vente,NaN,2020-11-20 01:08:41+00:00,2020,11,4,1,Land,Land,Batna,5
184531,batna-algerie-terrain-vente,0.0,MILLION,Batna,NaN,vente,NaN,2020-11-20 01:08:37+00:00,2020,11,4,1,Land,Land,Batna,5
184532,sidi-bel-abbes-algerie-appartement-vente-f4,9500000.0,MILLION,Sidi bel abbes,NaN,vente,NaN,2020-11-20 00:58:59+00:00,2020,11,4,0,Apartment,Apartment,Sidi Bel Abbes,22


In [27]:
df.to_csv('../data/processed/immobilier_cleaned.csv', index=False)

### **Seperate Residental and commercial use properties**

In [28]:
# List of residential property types we want to keep
RESIDENTIAL_TYPES = ['Apartment', 'Villa', 'House/Duplex', 'Bungalow']
COMMERCIAL_TYPES = ['Commercial', 'Industrial/Hangar', 'Building', 'Land']
# Filter the dataframe
df_residential = df[df['property_type_robust'].isin(RESIDENTIAL_TYPES)].copy()
df_commercial = df[df['property_type_robust'].isin(COMMERCIAL_TYPES)].copy()
# Optional: reset the index
df_residential.reset_index(drop=True, inplace=True)
df_commercial.reset_index(drop=True, inplace=True)
# Inspect
print(f"Original dataframe: {len(df)} listings")
print(f"Residential listings: {len(df_residential)} listings")
print(df_residential['property_type_robust'].value_counts())
print(f"Commercial listings: {len(df_commercial)} listings")
print(df_commercial['property_type_robust'].value_counts())
df_residential.to_csv('../data/processed/immobilier_residential_only.csv', index=False)
df_commercial.to_csv('../data/processed/immobilier_commercial_only.csv', index=False)

Original dataframe: 184534 listings
Residential listings: 120805 listings
property_type_robust
Apartment       85538
Villa           31944
House/Duplex     2671
Bungalow          652
Name: count, dtype: int64
Commercial listings: 57419 listings
property_type_robust
Land                 27777
Commercial           22245
Industrial/Hangar     4527
Building              2870
Name: count, dtype: int64


### **Splitting Listings by Room Count**

This dataset contains multiple apartment types inside the same publication  
(e.g., “F3–F4 promotion”). These introduce noise in any price/m² or valuation study,  
because one price and one area represent multiple different units.

To maintain analytical accuracy, we split the dataset into two subsets:

#### 1. **Single-Apartment Listings (`residential_single_unit.csv`)**
- Listings where the slug contains **exactly one room type** (e.g., F3 or F4).
- Clean `rooms` column is assigned from the detected room type.
- Used for:
  - Market valuation  
  - Price/m² analysis  
  - Predictive modeling  
  - Time-series and trend analysis  

These rows represent **true individual properties**, ideal for modeling real prices.

#### 2. **Multi-Apartment Listings (`residential_multi_unit.csv`)**
- Slugs containing **multiple room types** (e.g., "F3-F4-F5").
- Usually promotional or bulk offers.
- These distort price/area signals and are excluded from core analysis.
- Kept separately for:
  - Studying developer marketing dynamics  
  - Understanding promotion structures  
  - Detecting multi-unit advertising patterns  

### Rationale
Removing mixed-room listings ensures:
- Correct price/m² calculations  
- Cleaner regression relationships (rooms → price)  
- More robust predictive models  
- Avoids treating ambiguous promotions as single units  

This separation is standard practice in real estate data cleaning and improves the  
reliability of market analytics.

In [45]:
# --- Extract all room numbers (F1, F2, F3…) from slug ---
def extract_all_rooms(slug):
    if pd.isna(slug):
        return []
    rooms = re.findall(r'f(\d{1,2})', slug.lower())
    return sorted({int(r) for r in rooms})  # unique + sorted

df['rooms_list'] = df['slug'].apply(extract_all_rooms)

# --- Split dataset based on number of distinct room types ---
df['rooms_list_len'] = df['rooms_list'].apply(len)

df_single = df[df['rooms_list_len'] == 1].copy()
df_multi  = df[df['rooms_list_len'] > 1].copy()

# Clean "rooms" column for single-unit listings
df_single['rooms'] = df_single['rooms_list'].str[0]


#remove unecessary columns 
df_single = df_single.drop(columns=['rooms_list_len','rooms_list'])
# --- Save datasets ---
df_single.to_csv('../data/processed/residential_single_unit.csv', index=False)
df_multi.to_csv('../data/processed/residential_multi_unit.csv', index=False)

# --- Quick summary ---
print("Total residential:", len(df))
print("Single-unit listings:", len(df_single))
print("Multi-unit/promotion listings:", len(df_multi))

print("\nRooms distribution (single listings only):")
print(df_single['rooms'].value_counts().sort_index())

Total residential: 184534
Single-unit listings: 77662
Multi-unit/promotion listings: 1751

Rooms distribution (single listings only):
rooms
1       281
2      9525
3     36225
4     23394
5      6029
6      1243
7       421
8       185
9        89
10       90
11       15
12       40
13        3
14        5
15       13
16        2
17        2
18        3
20        9
21        1
22        2
23       28
24        1
25        1
27        1
33        4
34       24
37        1
38        1
39        1
40        1
41        1
42        1
43        2
44        1
45        6
50        1
54        2
55        2
67        1
78        2
82        1
95        1
98        1
Name: count, dtype: int64
